In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gvar as gv
import pprint

In [2]:
import re
def parse_data(input_data):
    data = {}

    baryon_order = ["N_E0", "Xi_E0", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
    baryon_mapping = {'lambda_z': 'Lambda_E0',
                      'proton': 'N_E0',
                      'sigma_p': 'Sigma_E0',
                      'sigma_star_p': 'Sigma*_E0',
                      'xi_star_z': 'Xi*_E0',
                      'xi_z': 'Xi_E0',
                      'delta_pp':'Delta_E0'}

    for key, value in input_data.items():
        ensemble_name, baryon = key

        best = value['best']
        avg = value['avg']

        number_best, uncertainty_best = gv.mean(gv.gvar(best)), gv.sdev(gv.gvar(best))
        number_avg, uncertainty_avg = gv.mean(gv.gvar(avg)), gv.sdev(gv.gvar(avg))

        if ensemble_name not in data:
            data[ensemble_name] = {'nolan_best':{},'nolan_avg':{}}

        new_baryon_name = baryon_mapping[baryon]
        
        data[ensemble_name]['nolan_best'][new_baryon_name] = (number_best, uncertainty_best)
        data[ensemble_name]['nolan_avg'] [new_baryon_name] = (number_avg, uncertainty_avg)
        
    for ensemble_name in data:
        for key in ['nolan_best','nolan_avg']:
            data[ensemble_name][key] = {baryon: data[ensemble_name][key].get(
                baryon, (None, None)) for baryon in baryon_order}

    return data


In [3]:
nolan = {}
nolan = gv.load('nolan.p')
nolan_ = parse_data(nolan)  
# pprint.pprint(nolan_)

In [4]:
import gvar
import re

def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  # If there are not enough lines, skip this table
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_').replace('$', '').replace('{', '').replace('}', '')
        
        data[ensemble] = {}

        for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
            cols = line.split("&")

            person = cols[0].strip()
            # Remove extra spaces within parentheses and then change the data to gvar
            results = [gv.gvar(item.replace(" ", "").replace("{", "").replace("}", "")) if "(" in item else (None, None) for item in cols[1:]]
            results_dict = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))
            sorted_results = sorted(results_dict.items(), key=lambda item: item[1][0] if isinstance(item[1], tuple) and item[1][0] is not None else (item[1] if isinstance(item[1], gvar._gvarcore.GVar) else float('inf')))
            data[ensemble][person] = sorted_results

    return data

file_name = "tables.txt"
# baryon_order = ["N_E0", "Xi_E0", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
baryon_order = ["N_E0", "Lambda_E0", "Sigma_E0","Xi_E0", "Delta_E0", "Sigma*_E0", "Xi*_E0"]
data = parse_latex_table(file_name)

for ensemble_name in data:
    nolan_best = {k : "{}({})".format(v[0], v[1]) if v[0] is not None and v[1] is not None else "None(None)" for k, v in nolan_[ensemble_name]['nolan_best'].items()}
    nolan_avg =  {k : "{}({})".format(v[0], v[1]) if v[0] is not None and v[1] is not None else "None(None)" for k, v in nolan_[ensemble_name]['nolan_avg'].items()}
    data[ensemble_name]['nolan_best'] = [(k, gvar.gvar(nolan_best[k])) for k in baryon_order if k in nolan_best and nolan_best[k] != "None(None)"]
    data[ensemble_name]['nolan_avg']  = [(k, gvar.gvar(nolan_avg[k])) for k in baryon_order if k in nolan_avg and nolan_avg[k] != "None(None)"]

# pprint.pprint(data)

In [5]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name, pdf):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(ensemble_name)

    offset = 0.06
    all_baryon_keys = []
    positions = {}

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline' or person == 'Jinchen':
            continue
        # valid_results = {k: v for k, v in results if len(v) > 1 and v[1] is not None}
        valid_results = {k: v for k, v in results if v is not None}

        # print(valid_results)

        x = np.arange(len(valid_results))
        y  = []
        for result in valid_results.items():
            try:
                y.append(gvar.mean(result[1]))
            except ValueError:
                y.append(np.nan)
        # y = [gv.mean(result[1]) for result in valid_results.items() if gv.mean(result[1]) is not None]
        def truncate(number, significant_figures):
            if number == 0:
                return 0
            else:
                return round(number, significant_figures - int(np.floor(np.log10(abs(number)))) - 1)

        # yerr = [truncate(result[1] * 0.001,2) if result[1][1] is not None else 0 for result in valid_results.items()]
        yerr = []
        for result in valid_results.items():
            try:
                yerr.append(gv.sdev(result[1]))
            except ValueError:
                yerr.append(np.nan)
        # yerr = [gv.sdev(result[1]) if gv.sdev(result[1]) is not None else 0 for result in valid_results.items()]
        # print(yerr)
        # ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)
        ax.errorbar(x + i * offset, y, yerr=np.array(yerr).T, fmt="o", capsize=5, label=person)

        # Store the positions of each baryon for this person
        for j, key in enumerate(valid_results.keys()):
            if key in positions:
                positions[key].append(j + i * offset)
            else:
                positions[key] = [j + i * offset]

        all_baryon_keys += [k for k in valid_results.keys() if k not in all_baryon_keys]

    mean_positions = [np.mean(positions[key]) for key in all_baryon_keys]
    xticks = np.arange(len(all_baryon_keys))

    ax.set_xticks(xticks)
    ax.set_xticklabels(all_baryon_keys, rotation=45, ha="right")

    # ax.set_xticks(mean_positions)
    # ax.set_xticklabels(all_baryon_keys, rotation=45,ha="right")
    ax.legend()
    pdf.savefig(fig, bbox_inches="tight")
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)


/usr/lib/python3.11/site-packages/matplotlib/axes/_axes.py:1185: RuntimeWarning: All-NaN axis encountered
  miny = np.nanmin(masked_verts[..., 1])
/usr/lib/python3.11/site-packages/matplotlib/axes/_axes.py:1186: RuntimeWarning: All-NaN axis encountered
  maxy = np.nanmax(masked_verts[..., 1])
